In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

### Конфигурации

In [30]:
# Задаем заголовки для нашего запроса
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36"
}

# Делаем GET запрос к сайту, используя указанные заголовки
html_text = requests.get(
    "https://www.transfermarkt.com/spieler-statistik/marktwertAenderungen/marktwertetop/?plus=1",
    headers=headers,
)

# Преобразуем полученный текст в объект BeautifulSoup для удобной работы с HTML
site = BeautifulSoup(html_text.content, "lxml")

# Ищем все div элементы с классом inline-select на странице
select_classes = site.find_all("div", class_="inline-select")

# Создаем пустой словарь для хранения информации о турнирах
tournaments_dict = {}

# Проходим по всем найденным div элементам
for select_class in select_classes:
    # Ищем все select элементы с атрибутом name равным wettbewerb_id
    find_tournaments = select_class.find("select", attrs={"name": "wettbewerb_id"})
    # Если такие элементы найдены, проходим по ним
    if find_tournaments is not None:
        # Ищем все элементы optgroup (группы опций в выпадающем списке)
        countries = find_tournaments.find_all("optgroup")
        for country in countries:
            # Обрабатываем группу "Все соревнования"
            if country.text == "\nAll Competitions\n":
                tournaments_dict["All"] = {"All": 0}
            else:
                # Иначе обрабатываем конкретное соревнование и записываем его в словарь
                country_name = country["label"]
                tournaments_dict[country_name] = {}
                tournaments = country.find_all("option")
                for tournament in tournaments:
                    tournaments_dict[country_name][tournament.text] = tournament[
                        "value"
                    ]
print(tournaments_dict)

# Создаем пустой словарь для хранения информации о странах
countries_dict = {}
for select_class in select_classes:
    # Ищем все select элементы c атрибутом name равным spieler_land_id
    find_countries = select_class.find("select", attrs={"name": "spieler_land_id"})
    if find_countries is not None:
        # Для каждой найденной страны добавляем ее в словарь, где ключ это название страны, а значение - ее код
        countries = find_countries.find_all("option")
        for country in countries:
            countries_dict[country.text] = int(country["value"])
countries_dict

# Словари для преобразования английских названий позиций игроков в немецкие аналоги и обратно
player_position = {
    "Goalkeeper": "Torwart",
    "Defender": "Abwehr",
    "Midfielder": "Mittelfeld",
    "Forward": "Sturm",
    "All": 0,
}
player_amplua = {
    "All": 0,
    "Goalkeeper": 1,
    "Sweeper": 2,
    "Centre-Back": 3,
    "Left-Back": 4,
    "Right-Back": 5,
    "Defensive Midfield": 6,
    "Central Midfield": 7,
    "Right Midfield": 8,
    "Left Midfield": 9,
    "Attacking Midfield": 10,
    "Left Winger": 11,
    "Right Winger": 12,
    "Second Striker": 13,
    "Centre-Forward": 14,
}

{'All': {'All': 0}, 'Egypt': {'Egyptian Premier League': 'EGY1'}, 'Albania': {'Abissnet Superiore': 'ALB1', 'Kategoria e Parë': 'ALB2', 'Kategoria Superiore U21': 'AL21'}, 'Algeria': {'Ligue Professionnelle 1': 'ALG1'}, 'Andorra': {'Primera Divisió': 'AND1'}, 'Angola': {'Girabola': 'AN1L'}, 'Argentina': {'Primera Nacional': 'ARG2', 'Professional Football League': 'AR1N', 'Copa de la Liga Profesional de Fútbol': 'CDLP'}, 'Armenia': {'Bardzraguyn khumb': 'ARM1', 'Armenian First League': 'ARM2'}, 'Australia': {'A-League Men': 'AUS1'}, 'Azerbaijan': {'Premyer Liqa': 'AZ1'}, 'Bangladesh': {'Bangladesh Premier League': 'BGD1'}, 'Belarus': {'Vysheyshaya Liga': 'WER1', 'Pershaya Liga': 'WER2'}, 'Belgium': {'Jupiler Pro League': 'BE1', 'Challenger Pro League': 'BE2'}, 'Bolivia': {'División Profesional Apertura': 'B1AP'}, 'Bosnia-Herzegovina': {'Premijer Liga': 'BOS1', 'Prva liga FBiH': 'BOS2', 'Prva Liga RS': 'BH2S'}, 'Brazil': {'Campeonato Brasileiro Série A': 'BRA1', 'Campeonato Brasileiro Sé

In [31]:
# Создаем список со всеми типами наград
allTypes = ["individual", "international", "national", "other"]

# Создаем список с названиями индивидуальных наград
individual = [
    "UEFA BEST PLAYER IN EUROPE",
    "FOOTBALLER OF THE YEAR",
    "FIFA BEST PLAYER",
    "TOP GOAL SCORER",
    "PLAYER OF THE YEAR",
    "TM-PLAYER OF THE SEASON",
]

# Создаем список с названиями международных наград
international = [
    "WORLD CUP WINNER",
    "CHAMPIONS LEAGUE WINNER",
    "UEFA SUPERCUP WINNER",
    "EUROPEAN CHAMPION CLUBS' CUP",
    "FIFA CLUB WORLD CUP WINNER",
    "UEFA CUP WINNER",
    "INTERCONTINENTAL CUP WINNER",
    "CUP WINNERS CUP WINNER",
]

# Создаем список с названиями национальных наград
national = [
    "ENGLISH CHAMPION",
    "ENGLISH FA CUP WINNER",
    "SPANISH CHAMPION",
    "SPANISH SUPER CUP WINNER",
    "SPANISH CUP WINNER",
    "GERMAN CUP WINNER",
    "GERMAN CHAMPION",
    "GERMAN SUPER CUP WINNER",
    "FRENCH SUPERCUP WINNER",
    "FRENCH CUP WINNER",
    "FRENCH CHAMPION",
    "FRENCH LEAGUE CUP WINNER",
    "ITALIAN CHAMPION",
    "ITALIAN CUP WINNER",
    "ITALIAN SUPER CUP WINNER",
]

In [32]:
# Создаем список с названиями сезонов
years = [
    "23/24",
    "22/23",
    "21/22",
    "20/21",
    "19/20",
    "18/19",
    "17/18",
    "16/17",
    "15/16",
    "14/15",
    "13/14",
]

## Определение функций

In [33]:
# Определение функции, которая принимает url и вызывает страницы на трансферном рынке игроков
def NationalTeamCareer(url):
    # Делаем GET-запрос к этому URL с заданными заголовками
    response = requests.get(url, headers=headers)

    # Преобразуем полученный ответ в объект BeautifulSoup
    soup = BeautifulSoup(response.text, "lxml")

    # Ищем раздел на странице, который содержит информацию о карьере игрока в национальной команде
    national_team_career_section = soup.find(
        "div", class_="box", attrs={"data-viewport": "Laenderspielkarriere"}
    )

    # Создаем пустой список для хранения данных о карьере игрока в национальной команде
    national_team_data = []
  
    # Если такой раздел найден, мы проходим по каждой строке в этом разделе
    if national_team_career_section:
        for row in national_team_career_section.find_all(
            "div", class_="national-career__row"
        ):
            # Получаем все ячейки в этой строке
            cells = row.find_all("div", class_="grid__cell")
            if cells:
                # Определяем статус игрока (играет, не играет, неизвестно)
                status_class = cells[0].get("class", [])
                status = (
                    "Playing"
                    if "national-career__cell--green" in status_class
                    else "Not Playing"
                    if "national-career__cell--red" in status_class
                    else "Unknown"
                )
  
                # Извлекаем уровень команды, дебют, количество матчей и голов
                team_level = cells[1].get_text(strip=True)
                debut = cells[2].get_text(strip=True)
                matches = cells[3].get_text(strip=True)
                goals = cells[4].get_text(strip=True)

                # Добавляем собранную информацию в список
                national_team_data.append(
                    {
                        "Status": status,
                        "Team Level": team_level,
                        "Debut": debut,
                        "Matches": matches,
                        "Goals": goals,
                    }
                )
  
    # Берем только последние данные (если они есть)
    latest_national_team_data = national_team_data[0] if national_team_data else {}
  
    # Определяем ссылку на профиль национального игрока
    national_player_profile_link = soup.find("a", text="Go to national player profile")
    national_player_profile_url = (
        f"https://www.transfermarkt.com{national_player_profile_link['href']}"
        if national_player_profile_link
        else "Not available"
    )

    # Возвращает последние данные и URL профиля национального игрока
    return latest_national_team_data, national_player_profile_url

In [34]:
# Функция для создания ссылки на базу данных игроков в соответствии с выбранными критериями
def cook_link(
    min_age,
    max_age,
    min_cost,
    max_cost,
    pos,
    amp,
    trnmt_country,
    trnmt,
    nation,
    club_country,
):
    # Строка ссылки на статистику игрока
    link_on_list = "https://www.transfermarkt.com/spieler-statistik/marktwertAenderungen/marktwertetop/?plus=1"

    # Получение кода позиции, амплуа, участия в турнире, национальности и страны клуба из соответствующих словарей
    position = player_position[pos]
    amplua = player_amplua[amp]
    participates_in_tournament = tournaments_dict[trnmt_country][trnmt]
    nationality = countries_dict[nation]
    clubs_country = countries_dict[club_country]

    # Добавление в ссылку соответствующих параметров
    link_on_list += f"&galerie=0"
    link_on_list += f"&position={str(position)}"
    link_on_list += f"&spielerposition_id={str(amplua)}"
    link_on_list += f"&spieler_land_id={str(nationality)}"
    link_on_list += f"&verein_land_id={str(clubs_country)}"
    link_on_list += f"&wettbewerb_id={str(participates_in_tournament)}"
    link_on_list += f"&alter={str(min_age)}+-+{str(max_age)}"
    link_on_list += f"&filtern_nach_alter={str(min_age)}%3B{str(max_age)}"
    link_on_list += f"&minAlter={str(min_age)}"
    link_on_list += f"&maxAlter={str(max_age)}"
    link_on_list += f"&minMarktwert={str(min_cost)}"
    link_on_list += f"&maxMarktwert={str(max_cost)}"
    link_on_list += "&yt0=%D0%9F%D0%BE%D0%BA%D0%B0%D0%B7%D0%B0%D1%82%D1%8C"

    # Возвращение результата
    return link_on_list


link_on_list = cook_link(
    16,
    20,
    "100.000",
    "50.000.000",
    "Forward",
    "All",
    "Germany",
    "Bundesliga",
    "Germany",
    "Germany",
)
print(link_on_list)

https://www.transfermarkt.com/spieler-statistik/marktwertAenderungen/marktwertetop/?plus=1&galerie=0&position=Sturm&spielerposition_id=0&spieler_land_id=40&verein_land_id=40&wettbewerb_id=L1&alter=16+-+20&filtern_nach_alter=16%3B20&minAlter=16&maxAlter=20&minMarktwert=100.000&maxMarktwert=50.000.000&yt0=%D0%9F%D0%BE%D0%BA%D0%B0%D0%B7%D0%B0%D1%82%D1%8C


In [35]:
# Определение функции, которая принимает url и возвращает данные о игроках
def get_players(link_on_list):
    # Фиксированный заголовок для нашего запроса
    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36"
    }

    # Делаем get запрос к этому URL с нашим фиксированным заголовком
    html_text = requests.get(link_on_list, headers=headers)

    # Преобразуем полученный текст в объект BeautifulSoup для работа с html
    site = BeautifulSoup(html_text.content, "lxml")

    # Получаем все игроки из списка
    list_of_players = site.find_all("tr", class_="odd") + site.find_all("tr", class_="even")

    # Инициализация словаря для хранения данных всех игроков
    players_dict = {}

    # Цикл по списку игроков
    for player_text in list_of_players:
        # Получаем всю информацию об игроке
        all_info = player_text.find_all("td", class_="zentriert") + player_text.find_all("td", class_="rechts")
        all_info += player_text.find_all("td", class_="rechts hauptlink") + player_text.find_all("td", class_="rechts hauptlink mwHoechstwertKarriere")
        player_box = player_text.find("table", class_="inline-table").find("td", class_="hauptlink")

        # Получаем ссылку на страницу игрока
        link_on_player_site = "https://www.transfermarkt.com" + player_box.find("a")["href"]
        
        # Получаем возраст, клуб и стоимость игрока
        player_age = all_info[1].text
        club_box = all_info[2].find("a")
        club_name = club_box["title"]
        club_link = "https://www.transfermarkt.com" + club_box["href"]

        current_market_value = all_info[5].find("b").text
        change_data = all_info[4].text
        old_market_value = all_info[6].text
        difference = all_info[7].text
        per_cent = all_info[3].text
        
        # Добавляем полученные данные в словарь
        player_dict = {
            "link": link_on_player_site,
            "age": player_age,
            "club": club_name,
            "clublink": club_link,
            "current value": current_market_value,
            "old value": old_market_value,
            "difference": difference,
            "change data": change_data,
        }
        players_dict[player_box.text] = player_dict
  
    # Возвращаем словарь с данными об игроках
    return players_dict

In [36]:
# Определение функции, которая принимает url и возвращает данные игрока
def ParseLink(url):
    # Делаем GET-запрос к этому URL с нашим фиксированным заголовком
    response = requests.get(url, headers=headers)

    # Преобразуем полученный ответ в объект BeautifulSoup
    soup = BeautifulSoup(response.text, "lxml")

    # Находим необходимую информацию на странице
    info = soup.find("div", class_="box viewport-tracking")
    s = str(BeautifulSoup(response.content, "html.parser"))
    index = s.find("Market value")

    # Получаем строку с рыночной ценой игрока
    text = s[index : index + 100]

    # Используем регулярное выражение для выделения цены из строки
    pattern = re.compile(r"€\d*\.?\d*[mk]")
    price = re.search(pattern, text).group()

    # Получаем все поля информации об игроке
    info_fields = info.findAll(
        "span",
        class_=[
            "info-tablecontent info-tablecontent--regular info-table__content--flex",
            "info-tablecontent info-tablecontent--regular",
        ],
    )

    # Получаем значения этих полей
    fields = [field.text.replace(":", "").strip() for field in info_fields]

    # Получаем ссылку на клуб игрока
    club_link = np.nan
    answers = []
    info_answers = info.findAll(
        "span",
        class_=[
            "info-tablecontent info-tablecontent--bold info-table__content--flex",
            "info-tablecontent info-tablecontent--bold",
        ],
    )

    # Получаем все ответы из этих полей
    answers = [" ".join(answer.text.split()) for answer in info_answers]

    # Получаем ссылку на клуб игрока
    club_link = info_answers[fields.index("Current club")].find("a")
    if club_link != None:
        club_link = "https://www.transfermarkt.com" + club_link["href"]
    else:
        club_link = np.nan

    # Создаем словарь с всей информацией об игроке
    data = dict(zip(fields, answers))

    # Если есть ссылки на социальные сети, добавляем их также
    if info.find("div", class_="socialmedia-icons") != None:
        socials = [
            (social.get("title"), social.get("href"))
            for social in info.find("div", class_="socialmedia-icons").findAll("a")
        ]
        data["Social-Media"] = socials
    else:
        data["Social-Media"] = [np.nan]

    data["link"] = url
    data["current_value"] = price
    data["club_link"] = club_link

    return data

In [37]:
# Определение функции, которая принимает url и имя игрока и возвращает датафрейм из одной строки с информацией об игроке
def get_player_info(url, name):
    # Получение информации об игроке
    data = ParseLink(url)

    # Достаем информацию об социальных медиа, если она есть
    if len(data["Social-Media"]) > 1:
        for media in data["Social-Media"]:
            if media[0] == "Instagram":
                data["Social-Media"] = [media[1]]
                break
        else:
            data["Social-Media"] = [media[1]]

    # Конвертация данных в DataFrame
    df = pd.DataFrame(data, index=[0])
    
    # Добавляем имя игрока
    df["player"] = name
    
    # Возвращаем полученный DataFrame
    return df

In [38]:
# Определение функции, которая принимает базовый url профиля игрока и список сезонов и возвращает статистику игрока по сезонам
def PlayerBasicStatsSeasons(player_profile_base, seasons):
    global headers

    # Определить позиции, которые нам нужны из таблицы статистики
    indexes = [1, 2, 4, 5, 6, 8, 9]
    keys = [
        "goals",
        "assists",
        "substitutions on",
        "substitutions off",
        "yellow cards",
        "red cards",
        "penalty goals",
        "minutes played",
        "appearances",
    ]

    stats_dict = {}
    stats_dict_each_tournament = {}

    # Цикл по каждому сезону в списке сезонов
    for season in seasons:
        stats_dict[season] = {}
        stats_dict_each_tournament[season] = {}

        # Инициализация счетчиков для каждого ключа
        for key in keys:
            stats_dict[season][key] = 0

        html_text = requests.get(
            player_profile_base + str(season) + "/plus/1#gesamt", headers=headers
        )
        player_site = BeautifulSoup(html_text.content, "lxml")

        # Получите все турниры, в которых участвовал игрок в этом сезоне
        tournaments = player_site.find_all("tr", class_="odd") + player_site.find_all(
            "tr", class_="even"
        )

        # Цикл по каждому турниру в списке турниров
        for i, tournament in enumerate(tournaments):
            tournament_title = tournament.find(
                "td", class_="hauptlink no-border-links"
            ).text
            tournament_info = tournament.find_all("td", class_="zentriert")
            tournament_timings = tournament.find_all("td", class_="rechts")

            # Добавить новый словарь для каждого турнира в этом сезоне
            stats_dict_each_tournament[season][tournament_title] = {}

            # Получать информацию из таблицы статистики
            for j, i in enumerate(indexes):
                char = tournament_info[i].text.replace(".", "")
                if char == "-":
                    stats_dict_each_tournament[season][tournament_title][keys[j]] = 0
                else:
                    stats_dict_each_tournament[season][tournament_title][keys[j]] = int(
                        char
                    )
                    stats_dict[season][keys[j]] += int(char)

            # Получить появления игрока и общее время игры
            appearances = tournament.find(
                "td", class_="zentriert player-profile-performance-data"
            ).text.replace(".", "")
            if appearances and appearances != "-":
                stats_dict[season]["appearances"] += int(appearances)
                stats_dict_each_tournament[season][tournament_title][
                    "appearances"
                ] = int(appearances)
            minutes_played = tournament_timings[-1].text[:-1].replace(".", "")
            if minutes_played and minutes_played != "-":
                stats_dict[season]["minutes played"] += int(minutes_played)
                stats_dict_each_tournament[season][tournament_title][
                    "minutes played"
                ] = int(minutes_played)

    return (stats_dict, stats_dict_each_tournament)

In [39]:
# Определение функции, которая принимает ссылку на профиль игрока и список сезонов и возвращает датафрейм с общей статистикой игрока по этим сезонам
def PlayerStatsSeasonsDF(link, seasons):
    # Получение статистики игрока по сезонам
    dictionaries = PlayerBasicStatsSeasons(link, seasons)
    for_each_season = dictionaries[0]

    # Инициализация словаря для статистики
    stats = {
        "appearances": [0],
        "goals": [0],
        "assists": [0],
        "substitutions on": [0],
        "substitutions off": [0],
        "yellow cards": [0],
        "red cards": [0],
        "penalty goals": [0],
        "minutes played": [0],
    }

    # Суммируем статистику по каждому сезону
    for key in for_each_season.keys():
        for statistic in for_each_season[key]:
            stats[statistic][0] += for_each_season[key][statistic]

    # Конвертируем словарь в датафрейм
    df = pd.DataFrame(data=stats)

    return df

# Тестирование функции
PlayerStatsSeasonsDF("https://www.transfermarkt.com/steven-bergwijn/leistungsdaten/spieler/284165/saison/",["2023"]).head()

,appearances,goals,assists,substitutions on,substitutions off,yellow cards,red cards,penalty goals,minutes played
0,23,8,5,0,7,5,0,5,1987


In [40]:
# Определение функции, которая преобразует ссылку на профиль игрока в ссылку на его статистику по сезонам
def transform_link(link):
    return link.replace("profil", "leistungsdaten") + "/saison/"

In [41]:
# Определение функции, которая возвращает список всех трофеев игрока исходя из ссылки на его профиль
def Trophies(url):
  # Заменить "profil" на "erfolge" в URL для доступа к странице трофеев игрока
  url = url.replace("profil", "erfolge")

  # Сделать запрос к странице трофеев, используя заданный заголовок
  response = requests.get(url, headers=headers)

  # Преобразование ответа в объект BeautifulSoup для дальнейшего анализа
  soup = BeautifulSoup(response.text, "lxml")

  # Найти все заголовки блоков с информацией о трофеях
  info = soup.findAll("h2", class_="content-box-headline")

  # Извлечь текст из каждого заголовка и убрать лишние пробелы
  trophies = [trophie.text.strip() for trophie in info][:-1]

  # Возврат полученного списка трофеев
  return trophies

In [42]:
# Определение функции, которая возвращает тип трофея на основе его названия
def TrophyType(trophie):
  # Преобразование названия трофея в верхний регистр для сравнения
  trophie = trophie.upper()

  # Возвращает тип трофея в зависимости от того, в какую категорию он попадает
  if trophie in individual:
    return "individual"
  elif trophie in international:
    return "international"
  elif trophie in national:
    return "national"
  else:
    return "other"

In [43]:
# Функция возвращающая количество кубков по каждому типу для конкретного игрока по ссылке на его профиль.
def TrophyCount(url):
    # Список трофеев игрока.
    trophies = Trophies(url)

    # Массив значений того, сколько раз был выигран трофей.
    count = list(map(int, [trophie.split()[0][:-1] for trophie in trophies]))

    # Массив с типом каждого трофея: индивидуальный, международный, национальный, иное.
    types = [TrophyType(" ".join(trophie.split()[1::]).upper()) for trophie in trophies]

    # Массив пар (количество, тип).
    typeCount = list(zip(types, count))

    # Возвращаем словарь {тип: количество}, суммируя по каждой паре.
    df = {
        trophyType: sum(pair[1] for pair in typeCount if pair[0] == trophyType)
        for trophyType in allTypes
    }
    return pd.DataFrame(data=df, index=[0])

In [44]:
# Определение функции, которая возвращает данные о карьере игрока в национальной команде, используя URL его профиля
def NationalTeamCareer(url):
  # Отправляем GET запрос на URL профиля игрока
  response = requests.get(url, headers=headers)
  # Преобразуем ответ в объект BeautifulSoup для анализа
  soup = BeautifulSoup(response.text, "lxml")

  # Инициализируем переменные для хранения данных о национальной команде и URL профиля национального игрока
  national_team_data = []
  national_player_profile_url = "Not available"

  # Ищем секцию карьеры в национальной команде на странице профиля игрока
  national_team_career_section = soup.find("div", class_="box", attrs={"data-viewport": "Laenderspielkarriere"})

  # Если секция найдена:
  if national_team_career_section:
    # Извлекаем все строки с данными о национальной команде
    for row in national_team_career_section.find_all("div", class_="national-career__row"):
      cells = row.find_all("div", class_="grid__cell")

      # Если ячейки найдены в строке:
      if cells:
        # Извлекаем статус игрока (если он играет - 2, если нет - 0)
        status_class = cells[0].get("class", [])
        status_code = 0 if "national-career__cell--red" in status_class else 2
        # Извлекаем название команды
        team_level = cells[1].get_text(strip=True)
        team_name = re.sub(r" U\d+$", "", team_level)
        # Если игрок уровня "U", помечаем его статус как 1
        if " U" in team_level:
          status_code = 1
        # Добавляем название команды и статус в список данных о национальной команде
        national_team_data.append(
          {
            "National Team": team_name,
            "National Status": status_code,
          }
        )

  # Если есть данные о национальной команде:
  if national_team_data:
    # Извлекаем последние данные о национальной команде
    latest_national_team_data = national_team_data[0]
    # Ищем ссылку на профиль национального игрока
    national_player_profile_link = soup.find("a", text="Go to national player profile")
    # Если ссылка найдена, извлекаем URL профиля национального игрока
    if national_player_profile_link and national_player_profile_link.get("href"):
      national_player_profile_url = f"https://www.transfermarkt.com{national_player_profile_link['href']}"
  # Если данных о национальной команде нет, устанавливаем "Unknown" и 0
  else:
    latest_national_team_data = {"National Team": "Unknown", "National Status": 0}

  # Возврат последних данных о национальной команде и URL профиля национального игрока
  return latest_national_team_data, national_player_profile_url

In [45]:
# Определение функции, которая извлекает статистику игрока в национальной команде
def scrape_national_team_stats(url):
  # Отправляем GET запрос на страницу игрока
  response = requests.get(url, headers=headers)
  # Преобразуем ответ в объект BeautifulSoup для анализа
  soup = BeautifulSoup(response.content, "html.parser")

  # Ищем секцию с названием команды
  team_name_section = soup.find("div", class_="dataName")
  # Если секция найдена, извлекаем название команды. Иначе, записываем "Unknown"
  team_name = (
      team_name_section.find("span", class_="hide-for-small").text.strip()
      if team_name_section
      else "Unknown"
  )

  # Ищем табличку со статистикой
  stats_table = soup.find("table", class_="items")
  player_stats = {}
  
  # Если табличка найдена:
  if stats_table:
    # Ищем футер, где отражены итоговые статистические данные
    footer = stats_table.find("tfoot")
    if footer:
      # Получаем две колонки с данными
      totals = footer.find("tr")
      cols = totals.find_all("td")
      # Если колонок достаточно для получения всех данных:
      if len(cols) >= 9:
        # Заполняем словарь со статистикой
        player_stats = {
            "National Matches": cols[2].text.strip(),
            "National Goals": cols[3].text.strip(),
            "National Assists": cols[4].text.strip(),
            "National Yellow Cards": cols[5].text.strip(),
            "National Red Cards": cols[7].text.strip(),
        }

  # Возвращаем словарь со статистикой
  return player_stats

In [46]:
# Определение функции, которая объединяет данные о карьере игрока в национальной команде и его статистику в национальной команде,
# преобразует их в формат DataFrame и возвращает его.
def get_national_data(url):
  # Вызов функции, которая возвращает данные о карьере игрока в национальной команде и URL его профиля
  national_team_career_data, profile_url = NationalTeamCareer(url)
  # Вызов функции, которая возвращает статистику игрока в национальной команде
  national_team_stats = scrape_national_team_stats(profile_url)

  # Объединение двух наборов данных в один
  combined_data = {national_team_career_data, national_team_stats}

  # Замена символа "-" на 0 в общих данных
  players_data = [combined_data]
  for player in players_data:
    for key, value in player.items():
      if value == "-":
        player[key] = 0

  # Преобразование получившегося набора данных в формат DataFrame и возвращение его
  return pd.DataFrame(players_data)

In [47]:
# Определение функции, которая возвращает информацию о клубе игрока на основе ссылки на его профиль в клубе
def get_club_info(url):
  # Отправляем GET запрос на URL профиля клуба
  response = requests.get(url, headers=headers)
  # Преобразуем ответ в объект BeautifulSoup для анализа
  soup = BeautifulSoup(response.text, "lxml")

  # Извлекаем лигу, в которой играет клуб
  league = soup.find("span", class_="data-header__club").find("a").text.strip()
  # Извлекаем стоимость клуба, если она доступна. Иначе устанавливаем "NaN"
  price = soup.find("a", class_="data-header__market-value-wrapper")
  if price:
    price = price.text.split()[0]
  else:
    price = "NaN"

  # Получаем URL страницы со статистикой клуба и отправляем на него GET запрос
  statistic_url = url.replace("startseite", "tabelle")
  response = requests.get(statistic_url, headers=headers)
  soup = BeautifulSoup(response.text, "lxml")

  # Извлекаем статистические данные клуба
  statistic = soup.find("tr", class_="table-highlight").findAll("td", class_="zentriert")[2:6]
  statistic = {
    "Matches": statistic[0].text,
    "Goals": statistic[1].text,
    "+/-": statistic[2].text,
    "Pts": statistic[3].text,
  }

  # Получаем URL страницы с трофеями клуба и отправляем на него GET запрос
  trophies_url = url.replace("startseite", "erfolge")
  response = requests.get(trophies_url, headers=headers)
  soup = BeautifulSoup(response.text, "lxml")

  # Извлекаем все трофеи клуба
  trophies = []
  for trophy in soup.findAll("h2", class_=""):
    trophies.append(trophy.text)
  
  # Извлекаем трофеи клуба за последние десять лет
  table = soup.find("div", class_="large-4 columns").findAll("td", class_="zentriert", style="")
  trophies_last_ten_years = []
  for year in table:
    if year.text in years:
      trophies_last_ten_years.append(year.next_sibling.next_sibling.next_sibling.next_sibling.text)
    else:
      break

  # Получаем URL страницы турнирного положения клуба и отправляем на него GET запрос
  ranking_url = url.replace("startseite", "platzierungen")
  response = requests.get(ranking_url, headers=headers)
  soup = BeautifulSoup(response.text, "lxml")

  # Извлекаем места клуба в лиге за последние десять лет
  league_rankings = []
  count = 0
  for rank in soup.findAll("td", class_="zentriert"):
    if count >= 11:
      break
    ranking = rank.find("b")
    if ranking:
      if count != 0:
        league_rankings.append(ranking.text)
      count += 1

  # Преобразуем собранные данные в формат DataFrame и возвращаем их
  return pd.DataFrame(
      {
        "club_league: ": league,
        "club_price: ": price,
        "club_statistics: ": str(statistic),
        "club_trophies: ": str(trophies),
        "trophies_in_recent_years: ": str(trophies_last_ten_years),
        "club_league_rankings: ": str(league_rankings),
      },
      index=[0],
  )

In [48]:
fields = ["season", "injury", "from", "until", "days", "games_missed"]


# Определение функции, которая возвращает информацию о травмах игрока на основе URL его профиля
def get_injuries(url):
  # Получаем URL страницы с травмами игрока
  url = url.replace("profil", "verletzungen")
  # Отправляем GET запрос на эту страницу
  response = requests.get(url, headers=headers)
  # Преобразуем ответ в объект BeautifulSoup для анализа
  soup = BeautifulSoup(response.text, "lxml")
  
  # Извлекаем все строки таблицы с травмами
  info = list(
    filter(
      lambda x: (x.find("td", class_=["hauptlink"]) != None),
      soup.findAll("tr", class_=["odd", "even"]),
    )
  )
  # Проверяем, является ли травма текущей
  current = [
    1 - (info_.find("td", class_="zentriert bg_rot_20") is None) for info_ in info
  ]
  # Извлекаем информацию из каждой строки
  info = list(map(lambda x: (x.findAll(string=True)[1::]), info))
  # Возвращаем список словарей с информацией о травмах
  data = [{field: data_ for (field, data_) in zip(fields, info_)} for info_ in info]
  df = pd.DataFrame(data)
  df["is_current"] = current

  # Преобразуем количество дней травмы в числовой формат
  try:
    df["days"] = df["days"].str.extract("(\d+)", expand=False).astype(int)
  except:
    df["days"] = np.nan

  # Агрегируем метрики травм
  final_df = pd.DataFrame()
  final_df["avg_injuries_per_season"] = [
    df.groupby("season")["days"].count().values.mean()
  ]
  final_df["avg_injury_duration"] = [df["days"].mean()]
  final_df["is_injured"] = [df["is_current"].max()]
  try:
    final_df["last_injury_date"] = [df.iloc[0]["until"]]
  except:
    final_df["last_injury_date"] = [np.nan]

  # Возвращаем DataFrame с агрегированными метриками травм
  return final_df

In [49]:
# Определение функции, которая возвращает историю рыночной стоимости игрока на основе ссылки на его профиль
def get_market_value_history(url):
  # Извлекаем id игрока из URL его профиля
  player_id = url.split("/")[-1]

  # Отправляем GET запрос на страницу графика изменения рыночной стоимости игрока
  response = requests.get(
    f"https://www.transfermarkt.us/ceapi/marketValueDevelopment/graph/{player_id}",
    headers=headers,
  ).json()

  # Инициализируем список для хранения истории рыночной стоимости игрока
  market_value_history = []

  # Для каждого значения в ответе:
  for cur_value in response["list"]:
    # Добавляем в историю значению рыночной стоимости и соответствующую ему дату
    market_value_history.append(
      {"value": cur_value["mw"], "date": cur_value["datum_mw"]}
    )

  # Возврат истории рыночной стоимости игрока
  return market_value_history

## Функции для парсинга раздела информации по игрокам

In [50]:
# Определение функции, которая генерирует DataFrame с подробной информацией о каждом игроке
def generate_details_df(links, names):
  # Инициализация пустого DataFrame
  details_df = pd.DataFrame()
  # Проход по всем ссылкам и именам игроков
  for i in range(len(links)): 
    try:
      # Получение информации о игроке и добавление его в общий DataFrame
      temp_df = get_player_info(links[i], names[i])
      temp_df["index"] = i
      details_df = pd.concat([details_df, temp_df])
    except:
      # Вывод информации об ошибке, если не удается получить информацию о игроке
      print(f"ERROR. Link: {links[i]} ; i={i}")
      continue
  # Установка индекса для общего DataFrame
  details_df.index = details_df["index"]
  # Возвращение общего DataFrame без столбца "index"
  return details_df.drop(columns=["index"])

In [51]:
# Определение функции, которая генерирует DataFrame с информацией о результативности каждого игрока
def generate_performance_df(links):
  # Инициализация пустого DataFrame
  performance_df = pd.DataFrame()
  
  # Проход по всем ссылкам на профили игроков
  for i in range(len(links)): 
    try:
      # Получение статистики игрока за сезоны 2023 и 2022, и добавление его в общий DataFrame
      temp_df = PlayerStatsSeasonsDF(transform_link(links[i]), ["2023", "2022"])
      temp_df["index"] = i
      performance_df = pd.concat([performance_df, temp_df])
    except:
      # Вывод информации об ошибке, если не удается получить статистику игрока
      print(f"ERROR. Link: {links[i]} ; i={i}")
      continue
  # Установка индекса для общего DataFrame
  performance_df.index = performance_df["index"]
  # Возвращение общего DataFrame без столбца "index"
  return performance_df.drop(columns=["index"])

In [52]:
# Определение функции, которая генерирует DataFrame с информацией о трофеях каждого игрока
def generate_trophies_df(links):
  # Инициализация пустого DataFrame
  trophies_df = pd.DataFrame()
  
  # Проход по всем ссылкам на профили игроков
  for i in range(len(links)): 
    try:
      # Получаем количество трофеев игрока и добавляем его в общий DataFrame
      temp_df = TrophyCount(links[i])
      temp_df["index"] = i
      trophies_df = pd.concat([trophies_df, temp_df])
    except:
      # Вывод информации об ошибке, если не удается получить количество трофеев игрока
      print(f"ERROR. Link: {links[i]} ; i={i}")
      continue
  # Устанавливаем индекс для общего DataFrame
  trophies_df.index = trophies_df["index"]
  # Возвращаем общий DataFrame без столбца "index"
  return trophies_df.drop(columns=["index"])

In [53]:
# Определение функции, которая генерирует DataFrame с информацией о национальной команде каждого игрока
def generate_national_df(links):
  # Инициализация пустого DataFrame
  national_df = pd.DataFrame()
  
  # Проход по всем ссылкам на профили игроков
  for i in range(len(links)): 
    try:
      # Получение информации о национальной команде игрока и добавление его в общий DataFrame
      temp_df = get_national_data(links[i])
      temp_df["index"] = i
      national_df = pd.concat([national_df, temp_df])
    except:
      # Вывод информации об ошибке, если не удается получить информацию о национальной команде игрока
      print(f"ERROR. Link: {links[i]} ; i={i}")
      continue
  # Устанавливаем индекс для общего DataFrame
  national_df.index = national_df["index"]
  # Возвращаем общий DataFrame без столбца "index"
  return national_df.drop(columns=["index"])

In [54]:
# Определение функции, которая генерирует DataFrame с информацией о травмах каждого игрока
def generate_injuries_df(links):
  # Инициализация пустого DataFrame
  injuries_df = pd.DataFrame()
  
  # Проход по всем ссылкам на профили игроков
  for i in range(len(links)): 
    try:
      # Получение информации о травмах игрока и добавление его в общий DataFrame
      temp_df = get_injuries(links[i])
      temp_df["index"] = i
      injuries_df = pd.concat([injuries_df, temp_df])
    except:
      # Вывод информации об ошибке, если не удается получить информацию о травмах игрока
      print(f"ERROR. Link: {links[i]} ; i={i}")
      continue
  # Уставновка индекса для общего DataFrame
  injuries_df.index = injuries_df["index"]
  # Возврат общего DataFrame без столбца "index"
  return injuries_df.drop(columns=["index"])

In [55]:
# Определение функции, которая генерирует DataFrame с информацией о клубах, в которых играют игроки
def generate_clubs_df(clublinks):
  # Инициализация пустого DataFrame
  clubs_df = pd.DataFrame()
  # Инициализация словаря для хранения информации о клубах
  club_dict = {}
  
  # Проход по всем ссылкам на профили клубов
  for i in range(len(clublinks)): 
    try:
      # Если информация о клубе уже была получена, то используем ее
      if clublinks[i] in club_dict:
        temp_df = club_dict[clublinks[i]]
        temp_df["index"] = i
      else:
        # В противном случае, получаем информацию о клубе и добавляем его в словарь
        temp_df = get_club_info(clublinks[i])
        temp_df["index"] = i
        club_dict[clublinks[i]] = temp_df

      # Добавляем информацию о клубе в общий DataFrame
      clubs_df = pd.concat([clubs_df, temp_df])
    except:
      # Вывод информации об ошибке, если не удается получить информацию о клубе
      print(f"ERROR. Link: {clublinks[i]} ; i={i}")
      continue
  # Установка индекса для общего DataFrame
  clubs_df.index = clubs_df["index"]
  # Возвращение общего DataFrame без столбца "index"
  return clubs_df.drop(columns=["index"])

In [56]:
# Определение функции, которая генерирует DataFrame с информацией об истории цен каждого игрока
def generate_price_history_df(links):
  # Инициализация пустого DataFrame
  price_df = pd.DataFrame()
  
  # Проход по всем ссылкам на профили игроков
  for i in range(len(links)): 
    try:
      # Получение истории цен игрока и добавление его в общий DataFrame
      temp_df = pd.DataFrame(
          [";".join([str(x) for x in get_market_value_history(links[i])])],
          columns=["pice_history"],
      )
      temp_df["index"] = i
      price_df = pd.concat([price_df, temp_df])
    except:
      # Вывод информации об ошибке, если не удается получить историю цен игрока
      print(f"ERROR. Link: {links[i]} ; i={i}")
      continue
  # Установка индекса для общего DataFrame
  price_df.index = price_df["index"]
  # Возврат общего DataFrame без столбца "index"
  return price_df.drop(columns=["index"])

# Запуск процесса парсинга

In [57]:
# Загружаем файл 'tier_1_links.csv'
filename = "tier_1_links.csv"
df = pd.read_csv(filename).drop(columns=["Unnamed: 0"])

# Получаем список ссылок и имен игроков
links = list(df["link"])
names = list(df.index.astype(str))

# выводим первые 5 записей DataFrame
df.head()

,link
0,https://www.transfermarkt.com/ederson/profil/s...
1,https://www.transfermarkt.com/scott-carson/pro...
2,https://www.transfermarkt.com/josko-gvardiol/p...
3,https://www.transfermarkt.com/nathan-ake/profi...
4,https://www.transfermarkt.com/sergio-gomez/pro...


In [34]:
df.iloc[0].link

'https://www.transfermarkt.com/ederson/profil/spieler/238223'

In [36]:
details_df = generate_details_df(links, names)
details_df.head(3)

ERROR. Link: https://www.transfermarkt.com/paul-pogba/profil/spieler/122153; i=1350
ERROR. Link: https://www.transfermarkt.com/papu-gomez/profil/spieler/20005; i=1501
ERROR. Link: https://www.transfermarkt.com/tim-kohler/profil/spieler/913429; i=1633
ERROR. Link: https://www.transfermarkt.com/max-moerstedt/profil/spieler/923595; i=1742
ERROR. Link: https://www.transfermarkt.com/seedy-jarju/profil/spieler/824353; i=1817
ERROR. Link: https://www.transfermarkt.com/frank-feller/profil/spieler/806948; i=1827
ERROR. Link: https://www.transfermarkt.com/madi-monamay/profil/spieler/937113; i=1844
ERROR. Link: https://www.transfermarkt.com/noah-engelbreth/profil/spieler/908951; i=1961
ERROR. Link: https://www.transfermarkt.com/max-wendt/profil/spieler/849390; i=2078
ERROR. Link: https://www.transfermarkt.com/senny-mayulu/profil/spieler/903693; i=2103
ERROR. Link: https://www.transfermarkt.com/ze-henrique/profil/spieler/1031409; i=2624
ERROR. Link: https://www.transfermarkt.com/andre-goncalves/pr

ERROR. Link: https://www.transfermarkt.com/aleksa-cvetkovic/profil/spieler/943877; i=7883
ERROR. Link: https://www.transfermarkt.com/pavle-ognjenovic/profil/spieler/851547; i=7885
ERROR. Link: https://www.transfermarkt.com/boston-billups/profil/spieler/1109812; i=7886
ERROR. Link: https://www.transfermarkt.com/stefan-bastaja/profil/spieler/941236; i=7896
ERROR. Link: https://www.transfermarkt.com/zeljko-arsic/profil/spieler/599258; i=7898
ERROR. Link: https://www.transfermarkt.com/luka-krstovic/profil/spieler/940599; i=7899
ERROR. Link: https://www.transfermarkt.com/vasilije-zunic/profil/spieler/833801; i=7934
ERROR. Link: https://www.transfermarkt.com/veljko-dimitrijevic/profil/spieler/848699; i=7949
ERROR. Link: https://www.transfermarkt.com/andreja-ristic/profil/spieler/846745; i=7962
ERROR. Link: https://www.transfermarkt.com/lazar-balevic/profil/spieler/943889; i=7967
ERROR. Link: https://www.transfermarkt.com/vukasin-jovanovic/profil/spieler/952767; i=8023
ERROR. Link: https://ww

ERROR. Link: https://www.transfermarkt.com/werick-caetano/profil/spieler/331376; i=10359
ERROR. Link: https://www.transfermarkt.com/federico-zanetti/profil/spieler/826315; i=10361
ERROR. Link: https://www.transfermarkt.com/nikolay-hristov/profil/spieler/671536; i=10372
ERROR. Link: https://www.transfermarkt.com/jean-marc-tiboue/profil/spieler/540658; i=10393
ERROR. Link: https://www.transfermarkt.com/marto-boychev/profil/spieler/1084760; i=10472
ERROR. Link: https://www.transfermarkt.com/diedry-kouassi/profil/spieler/749200; i=10475
ERROR. Link: https://www.transfermarkt.com/ehije-ukaki/profil/spieler/1225900; i=10493
ERROR. Link: https://www.transfermarkt.com/aleksandar-veselinski/profil/spieler/1139960; i=10519
ERROR. Link: https://www.transfermarkt.com/weslen-junior/profil/spieler/690824; i=10535
ERROR. Link: https://www.transfermarkt.com/boyan-zagorski/profil/spieler/984603; i=10563
ERROR. Link: https://www.transfermarkt.com/ivan-dishkov/profil/spieler/894087; i=10565
ERROR. Link: 

ERROR. Link: https://www.transfermarkt.com/arman-sutkovic/profil/spieler/654460; i=13127
ERROR. Link: https://www.transfermarkt.com/arman-dzankovic/profil/spieler/652757; i=13200
ERROR. Link: https://www.transfermarkt.com/anton-putilo/profil/spieler/59491; i=13235
ERROR. Link: https://www.transfermarkt.com/yan-martinkevich/profil/spieler/1231547; i=13289
ERROR. Link: https://www.transfermarkt.com/danila-garbuz/profil/spieler/1167302; i=13294
ERROR. Link: https://www.transfermarkt.com/rodion-medvedev/profil/spieler/981863; i=13301
ERROR. Link: https://www.transfermarkt.com/ivan-khovalko/profil/spieler/898349; i=13307
ERROR. Link: https://www.transfermarkt.com/pavel-shevchenko/profil/spieler/981860; i=13309
ERROR. Link: https://www.transfermarkt.com/yuriy-kravchenko/profil/spieler/1149478; i=13311
ERROR. Link: https://www.transfermarkt.com/aleksandr-butko/profil/spieler/351725; i=13473
ERROR. Link: https://www.transfermarkt.com/boni-amian/profil/spieler/1217375; i=13523
ERROR. Link: http

ERROR. Link: https://www.transfermarkt.com/eero-vuorjoki/profil/spieler/1073022; i=15111
ERROR. Link: https://www.transfermarkt.com/oliver-heino/profil/spieler/905439; i=15145
ERROR. Link: https://www.transfermarkt.com/nicolas-gianini-dantas/profil/spieler/685803; i=15147
ERROR. Link: https://www.transfermarkt.com/liam-hughes/profil/spieler/494284; i=15154
ERROR. Link: https://www.transfermarkt.com/tuukka-torma/profil/spieler/942017; i=15161
ERROR. Link: https://www.transfermarkt.com/pedro-machado/profil/spieler/598542; i=15166
ERROR. Link: https://www.transfermarkt.com/emmanuel-okereke/profil/spieler/1225388; i=15168
ERROR. Link: https://www.transfermarkt.com/wille-nunez/profil/spieler/1235140; i=15174
ERROR. Link: https://www.transfermarkt.com/lucas-vayrynen/profil/spieler/945334; i=15186
ERROR. Link: https://www.transfermarkt.com/vincent-ulundu/profil/spieler/1051013; i=15195
ERROR. Link: https://www.transfermarkt.com/arttu-tulehmo/profil/spieler/1018044; i=15208
ERROR. Link: https:

...

ERROR. Link: https://www.transfermarkt.com/francesco-pari/profil/spieler/714068; i=19191
ERROR. Link: https://www.transfermarkt.com/antonio-guancia/profil/spieler/1223618; i=19192
ERROR. Link: https://www.transfermarkt.com/ariljo-hoxha/profil/spieler/979223; i=19193
ERROR. Link: https://www.transfermarkt.com/tommaso-argenziano/profil/spieler/1184925; i=19194
ERROR. Link: https://www.transfermarkt.com/tommaso-nicolini/profil/spieler/1185833; i=19196
ERROR. Link: https://www.transfermarkt.com/giuseppe-monteleone/profil/spieler/968456; i=19197
ERROR. Link: https://www.transfermarkt.com/richard-ulloa/profil/spieler/1234827; i=19198
ERROR. Link: https://www.transfermarkt.com/luca-magnani/profil/spieler/1192426; i=19199
ERROR. Link: https://www.transfermarkt.com/nicholas-di-meo/profil/spieler/930115; i=19201
ERROR. Link: https://www.transfermarkt.com/klajdi-buzi/profil/spieler/1227165; i=19202
ERROR. Link: https://www.transfermarkt.com/emanuele-polizzi/profil/spieler/1223305; i=19203
ERROR. 

ERROR. Link: https://www.transfermarkt.com/luis-gabriel-santos-silva/profil/spieler/1112249; i=19289
ERROR. Link: https://www.transfermarkt.com/martin-cremona/profil/spieler/374790; i=19290
ERROR. Link: https://www.transfermarkt.com/paul-rapa/profil/spieler/246762; i=19291
ERROR. Link: https://www.transfermarkt.com/joshua-buttigieg/profil/spieler/497741; i=19292
ERROR. Link: https://www.transfermarkt.com/andre-azzopardi/profil/spieler/501296; i=19293
ERROR. Link: https://www.transfermarkt.com/hayden-muscat/profil/spieler/863107; i=19294
ERROR. Link: https://www.transfermarkt.com/leo-coca/profil/spieler/723473; i=19295
ERROR. Link: https://www.transfermarkt.com/matias-camilleri-psaila/profil/spieler/987739; i=19297
ERROR. Link: https://www.transfermarkt.com/andre-martins/profil/spieler/643203; i=19298
ERROR. Link: https://www.transfermarkt.com/charlon-muscat/profil/spieler/515365; i=19299
ERROR. Link: https://www.transfermarkt.com/gaetano-bugeja/profil/spieler/849143; i=19300
ERROR. Lin

ERROR. Link: https://www.transfermarkt.com/savio-galea/profil/spieler/579422; i=19385
ERROR. Link: https://www.transfermarkt.com/fabian-xuereb/profil/spieler/628099; i=19386
ERROR. Link: https://www.transfermarkt.com/john-saliba/profil/spieler/516951; i=19387
ERROR. Link: https://www.transfermarkt.com/nathan-theuma/profil/spieler/579423; i=19388
ERROR. Link: https://www.transfermarkt.com/luke-sultana/profil/spieler/869087; i=19389
ERROR. Link: https://www.transfermarkt.com/matthias-cordina/profil/spieler/985533; i=19390
ERROR. Link: https://www.transfermarkt.com/kasem-darwish/profil/spieler/1213021; i=19391
ERROR. Link: https://www.transfermarkt.com/christian-debono/profil/spieler/253008; i=19392
ERROR. Link: https://www.transfermarkt.com/michael-paul-camilleri/profil/spieler/849206; i=19393
ERROR. Link: https://www.transfermarkt.com/marquinho/profil/spieler/364460; i=19394
ERROR. Link: https://www.transfermarkt.com/simon-galea/profil/spieler/238954; i=19395
ERROR. Link: https://www.tr

ERROR. Link: https://www.transfermarkt.com/michael-xuereb/profil/spieler/1144589; i=19479
ERROR. Link: https://www.transfermarkt.com/christian-bugeja/profil/spieler/246683; i=19480
ERROR. Link: https://www.transfermarkt.com/john-grech/profil/spieler/372752; i=19481
ERROR. Link: https://www.transfermarkt.com/brian-parada/profil/spieler/998406; i=19482
ERROR. Link: https://www.transfermarkt.com/julian-vella/profil/spieler/1193637; i=19483
ERROR. Link: https://www.transfermarkt.com/hayden-bonnici/profil/spieler/1193329; i=19484
ERROR. Link: https://www.transfermarkt.com/luke-vella/profil/spieler/372746; i=19485
ERROR. Link: https://www.transfermarkt.com/carlo-abela/profil/spieler/627498; i=19486
ERROR. Link: https://www.transfermarkt.com/joseph-azzopardi/profil/spieler/246673; i=19487
ERROR. Link: https://www.transfermarkt.com/john-vella/profil/spieler/246786; i=19488
ERROR. Link: https://www.transfermarkt.com/kevin-farrugia/profil/spieler/278670; i=19489
ERROR. Link: https://www.transfer

,Full name,Date of birth/Age,Place of birth,Height,Citizenship,Position,Foot,Player agent,Current club,Joined,...,club_link,player,Name in home country,Last contract extension,Contract option,2nd club,On loan from,Contract there expires,3nd club,4nd club
index,,,,,,,,,,,,,,,,,,,,,
0,Ederson Santana de Moraes,"Aug 17, 1993 (30)",Osasco (SP),"1,88 m",Brazil Portugal,Goalkeeper,left,Gestifute,Manchester City,"Jul 1, 2017",...,https://www.transfermarkt.com/manchester-city/...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,"Sep 3, 1985 (38)",Whitehaven,"1,88 m",England,Goalkeeper,right,Wasserman,Manchester City,"Jul 20, 2021",...,https://www.transfermarkt.com/manchester-city/...,1,Scott Paul Carson,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,"Jan 23, 2002 (22)",Zagreb,"1,85 m",Croatia,Defender - Centre-Back,left,Marjan Sisic,Manchester City,"Aug 5, 2023",...,https://www.transfermarkt.com/manchester-city/...,2,Joško Gvardiol,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
# Сохранение details_df в файл Excel
details_df.to_excel("details.xlsx", index=True)

In [63]:
# Генерация DataFrame с информацией о производительности каждого игрока 
performance_df = generate_performance_df(links)

# Показ первых 5 записей performance_df
performance_df.head()

,appearances,goals,assists,substitutions on,substitutions off,yellow cards,red cards,penalty goals,minutes played
index,,,,,,,,,
0,81,0,0,1,9,0,67,31,7162
1,0,0,0,0,0,0,0,0,0
2,68,3,1,11,11,5,1,0,5146
3,75,6,2,16,13,3,0,0,5312
4,35,0,2,21,5,0,1,0,1342


In [64]:
# Сохранение performance_df в файл Excel
performance_df.to_excel("performance.xlsx", index=True)

In [31]:
# Генерация DataFrame с информацией о трофеях каждого игрока 
trophies_df = generate_trophies_df(links)

# Показ первых 3 записей trophies_df
trophies_df.head(3)

,individual,international,national,other
index,,,,
0,0,3,7,12
1,0,4,4,3
2,0,2,2,4


In [32]:
# Сохранение trophies_df в файл Excel
trophies_df.to_excel("trophies.xlsx", index=True)

In [33]:
# Генерация DataFrame с информацией о национальных трофеях каждого игрока
national_df = generate_national_df(links)

# Показ первых 3 записей national_df
national_df.head(3)

ERROR. Link: https://www.transfermarkt.com/deivid-washington/profil/spieler/1082850; i=37
ERROR. Link: https://www.transfermarkt.com/alfie-gilchrist/profil/spieler/790992; i=43
ERROR. Link: https://www.transfermarkt.com/dan-burn/profil/spieler/134270; i=84
ERROR. Link: https://www.transfermarkt.com/sean-longstaff/profil/spieler/346707; i=88
ERROR. Link: https://www.transfermarkt.com/mark-gillespie/profil/spieler/142389; i=94
ERROR. Link: https://www.transfermarkt.com/igor/profil/spieler/380350; i=109
ERROR. Link: https://www.transfermarkt.com/carlos-baleba/profil/spieler/973085; i=126
ERROR. Link: https://www.transfermarkt.com/ryan-trevitt/profil/spieler/874538; i=144
ERROR. Link: https://www.transfermarkt.com/murillo/profil/spieler/1005649; i=166
ERROR. Link: https://www.transfermarkt.com/ryan-yates/profil/spieler/425902; i=188
ERROR. Link: https://www.transfermarkt.com/beto/profil/spieler/595809; i=217
ERROR. Link: https://www.transfermarkt.com/daniel-bentley/profil/spieler/136401; i

ERROR. Link: https://www.transfermarkt.com/xeber-alkain/profil/spieler/572271; i=785
ERROR. Link: https://www.transfermarkt.com/samuel-lino/profil/spieler/694928; i=835
ERROR. Link: https://www.transfermarkt.com/toni-fuidias/profil/spieler/623526; i=840
ERROR. Link: https://www.transfermarkt.com/david-lopez/profil/spieler/129444; i=841
ERROR. Link: https://www.transfermarkt.com/juanpe/profil/spieler/115333; i=842
ERROR. Link: https://www.transfermarkt.com/juan-carlos/profil/spieler/120705; i=851
ERROR. Link: https://www.transfermarkt.com/toni-villa/profil/spieler/337066; i=859
ERROR. Link: https://www.transfermarkt.com/sergi-altimira/profil/spieler/466801; i=869
ERROR. Link: https://www.transfermarkt.com/aitor-ruibal/profil/spieler/378139; i=873
ERROR. Link: https://www.transfermarkt.com/chimy-avila/profil/spieler/167721; i=874
ERROR. Link: https://www.transfermarkt.com/marcao/profil/spieler/439022; i=890
ERROR. Link: https://www.transfermarkt.com/joan-jordan/profil/spieler/263727; i=8

ERROR. Link: https://www.transfermarkt.com/mohamed-kaba/profil/spieler/911328; i=1556
ERROR. Link: https://www.transfermarkt.com/remi-oudin/profil/spieler/399543; i=1558
ERROR. Link: https://www.transfermarkt.com/mattia-chiesa/profil/spieler/535881; i=1564
ERROR. Link: https://www.transfermarkt.com/charlys/profil/spieler/1082519; i=1569
ERROR. Link: https://www.transfermarkt.com/michael-folorunsho/profil/spieler/377042; i=1571
ERROR. Link: https://www.transfermarkt.com/tijjani-noslin/profil/spieler/457736; i=1574
ERROR. Link: https://www.transfermarkt.com/thomas-henry/profil/spieler/412513; i=1575
ERROR. Link: https://www.transfermarkt.com/juan-manuel-cruz/profil/spieler/748316; i=1576
ERROR. Link: https://www.transfermarkt.com/alessandro-berardi/profil/spieler/127217; i=1578
ERROR. Link: https://www.transfermarkt.com/giangiacomo-magnani/profil/spieler/310458; i=1579
ERROR. Link: https://www.transfermarkt.com/fabien-centonze/profil/spieler/384859; i=1581
ERROR. Link: https://www.transf

ERROR. Link: https://www.transfermarkt.com/abdoul-kader-bamba/profil/spieler/560725; i=2271
ERROR. Link: https://www.transfermarkt.com/brendan-chardonnet/profil/spieler/250779; i=2276
ERROR. Link: https://www.transfermarkt.com/kenny-lala/profil/spieler/174196; i=2278
ERROR. Link: https://www.transfermarkt.com/hugo-magnetti/profil/spieler/474662; i=2281
ERROR. Link: https://www.transfermarkt.com/axel-camblan/profil/spieler/924800; i=2282
ERROR. Link: https://www.transfermarkt.com/jeremy-le-douaron/profil/spieler/737781; i=2283
ERROR. Link: https://www.transfermarkt.com/adrien-lebeau/profil/spieler/395674; i=2284
ERROR. Link: https://www.transfermarkt.com/yan-marillat/profil/spieler/454149; i=2286
ERROR. Link: https://www.transfermarkt.com/antonin-cartillier/profil/spieler/774772; i=2289
ERROR. Link: https://www.transfermarkt.com/julien-le-cardinal/profil/spieler/794349; i=2290
ERROR. Link: https://www.transfermarkt.com/pierre-lees-melou/profil/spieler/377393; i=2291
ERROR. Link: https:/

ERROR. Link: https://www.transfermarkt.com/lucas-paes/profil/spieler/407841; i=2753
ERROR. Link: https://www.transfermarkt.com/pablo-roberto/profil/spieler/592375; i=2761
ERROR. Link: https://www.transfermarkt.com/nuno-moreira/profil/spieler/461937; i=2763
ERROR. Link: https://www.transfermarkt.com/andre-lacximicant/profil/spieler/752442; i=2764
ERROR. Link: https://www.transfermarkt.com/samu/profil/spieler/1159449; i=2766
ERROR. Link: https://www.transfermarkt.com/felippe-cardoso/profil/spieler/596067; i=2767
ERROR. Link: https://www.transfermarkt.com/fahem-benaissa-yahia/profil/spieler/805864; i=2772
ERROR. Link: https://www.transfermarkt.com/gaizka-larrazabal/profil/spieler/439465; i=2773
ERROR. Link: https://www.transfermarkt.com/neto/profil/spieler/291968; i=2777
ERROR. Link: https://www.transfermarkt.com/fernando-andrade/profil/spieler/236537; i=2778
ERROR. Link: https://www.transfermarkt.com/kiki/profil/spieler/529730; i=2780
ERROR. Link: https://www.transfermarkt.com/clayton/pr

...

ERROR. Link: https://www.transfermarkt.com/patrick-anzaghi/profil/spieler/449313; i=19114
ERROR. Link: https://www.transfermarkt.com/francesco-sartori/profil/spieler/163472; i=19115
ERROR. Link: https://www.transfermarkt.com/luigi-renna/profil/spieler/1228276; i=19117
ERROR. Link: https://www.transfermarkt.com/fabio-cateni/profil/spieler/938344; i=19118
ERROR. Link: https://www.transfermarkt.com/maurizio-nicola-bruno/profil/spieler/1185164; i=19119
ERROR. Link: https://www.transfermarkt.com/luca-simeoni/profil/spieler/616916; i=19121
ERROR. Link: https://www.transfermarkt.com/william-edoardo-garcia/profil/spieler/813021; i=19122
ERROR. Link: https://www.transfermarkt.com/francesco-gori/profil/spieler/796780; i=19123
ERROR. Link: https://www.transfermarkt.com/gennaro-capparelli/profil/spieler/934209; i=19124
ERROR. Link: https://www.transfermarkt.com/angelo-russo/profil/spieler/650962; i=19125
ERROR. Link: https://www.transfermarkt.com/marzio-pica/profil/spieler/625088; i=19126
ERROR. L

ERROR. Link: https://www.transfermarkt.com/brandon-said/profil/spieler/256970; i=19252
ERROR. Link: https://www.transfermarkt.com/gaetano-bujega/profil/spieler/498545; i=19253
ERROR. Link: https://www.transfermarkt.com/saviour-attard/profil/spieler/253000; i=19254
ERROR. Link: https://www.transfermarkt.com/paul-camilleri/profil/spieler/369175; i=19255
ERROR. Link: https://www.transfermarkt.com/kurt-grima/profil/spieler/516246; i=19256
ERROR. Link: https://www.transfermarkt.com/junior-sergipe/profil/spieler/1096912; i=19257
ERROR. Link: https://www.transfermarkt.com/predrag-djorjevic/profil/spieler/477254; i=19258
ERROR. Link: https://www.transfermarkt.com/tristen-portelli/profil/spieler/627502; i=19259
ERROR. Link: https://www.transfermarkt.com/marcelo-barbosa/profil/spieler/627499; i=19262
ERROR. Link: https://www.transfermarkt.com/ian-xuereb/profil/spieler/246791; i=19263
ERROR. Link: https://www.transfermarkt.com/joseph-grima/profil/spieler/246719; i=19265
ERROR. Link: https://www.t

ERROR. Link: https://www.transfermarkt.com/marco-ciangura/profil/spieler/997689; i=19354
ERROR. Link: https://www.transfermarkt.com/johandre-padilla/profil/spieler/1102164; i=19355
ERROR. Link: https://www.transfermarkt.com/kenny-hili/profil/spieler/412859; i=19356
ERROR. Link: https://www.transfermarkt.com/matthew-portelli/profil/spieler/296460; i=19357
ERROR. Link: https://www.transfermarkt.com/ismael-belhadj/profil/spieler/965734; i=19358
ERROR. Link: https://www.transfermarkt.com/dylan-paul-rapa/profil/spieler/988858; i=19359
ERROR. Link: https://www.transfermarkt.com/damon-hili/profil/spieler/478690; i=19360
ERROR. Link: https://www.transfermarkt.com/claudio-murici/profil/spieler/682048; i=19361
ERROR. Link: https://www.transfermarkt.com/emmanuel-godwin/profil/spieler/1193685; i=19362
ERROR. Link: https://www.transfermarkt.com/luke-buttigieg/profil/spieler/478686; i=19363
ERROR. Link: https://www.transfermarkt.com/joshua-micallef/profil/spieler/1032251; i=19364
ERROR. Link: https:

ERROR. Link: https://www.transfermarkt.com/christian-bartolo/profil/spieler/1082664; i=19449
ERROR. Link: https://www.transfermarkt.com/john-rapa/profil/spieler/288333; i=19450
ERROR. Link: https://www.transfermarkt.com/armando-xerri/profil/spieler/587791; i=19451
ERROR. Link: https://www.transfermarkt.com/christian-cardona/profil/spieler/517392; i=19452
ERROR. Link: https://www.transfermarkt.com/joao-ferrari/profil/spieler/965150; i=19453
ERROR. Link: https://www.transfermarkt.com/simon-spiteri/profil/spieler/587788; i=19454
ERROR. Link: https://www.transfermarkt.com/joseph-buttigieg/profil/spieler/587789; i=19455
ERROR. Link: https://www.transfermarkt.com/christian-vella/profil/spieler/863123; i=19456
ERROR. Link: https://www.transfermarkt.com/adrian-vella/profil/spieler/288341; i=19457
ERROR. Link: https://www.transfermarkt.com/sintayehu-curmi/profil/spieler/498069; i=19458
ERROR. Link: https://www.transfermarkt.com/jamie-xerri/profil/spieler/1081725; i=19459
ERROR. Link: https://ww

,National Team,National Status,National Matches,National Goals,National Assists,National Yellow Cards,National Red Cards
index,,,,,,,
0,Brazil,2,25,0,2,0,18
1,England,0,4,0,0,0,4
2,Croatia,2,27,2,0,2,0


In [34]:
# Сохранение national_df в файл Excel
national_df.to_excel("national.xlsx", index=True)

In [ ]:
# Генерация DataFrame с информацией о травмах каждого игрока
injuries_df = generate_injuries_df(links)

# Показ первых 3 записей injuries_df
injuries_df.head(3)

In [ ]:
# Сохранение injuries_df в файл Excel
injuries_df.to_excel("injuries.xlsx", index=True)

In [ ]:
# Генерация DataFrame с информацией о клубах, перечисленных в details_df
clubs_df = generate_clubs_df(details_df["club_link"])

# Показ первых 3 записей clubs_df
clubs_df.head(3)

In [ ]:
# Сохранение clubs_df в файл Excel
clubs_df.to_excel("clubs.xlsx", index=True)

In [111]:
# Генерация DataFrame с историей цен каждого игрока
price_history_df = generate_price_history_df(df["link"])

# Показ первых 5 записей price_history_df
price_history_df.head()

ERROR. Link: https://www.transfermarkt.com/carlos-vinicius/profil/spieler/525287 ; i=3564
ERROR. Link: https://www.transfermarkt.com/lucas-torreira/profil/spieler/318077 ; i=3570
ERROR. Link: https://www.transfermarkt.com/igor-akinfeev/profil/spieler/15570 ; i=4844
ERROR. Link: https://www.transfermarkt.com/ilya-agapov/profil/spieler/582232 ; i=4846
ERROR. Link: https://www.transfermarkt.com/ruben-tigran-yesayan/profil/spieler/623463 ; i=14271


,price_history
index,
0,"{'value': '€300k', 'date': 'Apr 3, 2013'};{'va..."
1,"{'value': '€400k', 'date': 'Oct 4, 2004'};{'va..."
2,"{'value': '€300k', 'date': 'Sep 16, 2019'};{'v..."
3,"{'value': '€1.50m', 'date': 'Jul 25, 2013'};{'..."
4,"{'value': '€2.00m', 'date': 'Apr 4, 2018'};{'v..."


# Объединение разделов информации об игроках

In [3]:
# Чтение файлов Excel и преобразование их в DataFrame
details_df = pd.read_excel("details.xlsx")
performance_df = pd.read_excel("performance.xlsx")
trophies_df = pd.read_excel("trophies.xlsx")
national_df = pd.read_excel("national.xlsx")
injuries_df = pd.read_excel("injuries.xlsx")
clubs_df = pd.read_excel("clubs.xlsx")
price_history_df = pd.read_excel("price.xlsx")

In [4]:
# Объединение всех DataFrame в один общий DataFrame
result = details_df
for df in [
    performance_df,
    trophies_df,
    national_df,
    injuries_df,
    clubs_df,
    price_history_df,
]:
    result = result.merge(df, on="index", how="left")

# Заполнение пропущенных значений в столбце 'Full name' значениями из столбца 'Name in home country'
result["Full name"] = result["Full name"].fillna(result["Name in home country"])

# Удаление ненужных столбцов 'Name in home country' и 'player'
result.drop(columns=["Name in home country", "player"], inplace=True)

# Показ первых 5 записей общего DataFrame
result.head()

,index,Full name,Date of birth/Age,Place of birth,Height,Citizenship,Position,Foot,Player agent,Current club,...,avg_injury_duration,is_injured,last_injury_date,club_league:,club_price:,club_statistics:,club_trophies:,trophies_in_recent_years:,club_league_rankings:,price_history
0,0,Ederson Santana de Moraes,"Aug 17, 1993 (30)",Osasco (SP),"1,88 m",Brazil Portugal,Goalkeeper,left,Gestifute,Manchester City,...,12.000000,0.0,"Jan 20, 2024",Premier League,€1.26bn,"{'Matches': '26', 'Goals': '59:26', '+/-': '33...","['1x Champions League Winner', '1x FIFA Club W...","['UEFA Supercup Winner', 'FIFA Club World Cup ...","['1', '1', '1', '2', '1', '1', '3', '4', '2', ...","{'value': '€300k', 'date': 'Apr 3, 2013'};{'va..."
1,1,Scott Paul Carson,"Sep 3, 1985 (38)",Whitehaven,"1,88 m",England,Goalkeeper,right,Wasserman,Manchester City,...,30.333333,0.0,"Mar 14, 2021",Premier League,€1.26bn,"{'Matches': '26', 'Goals': '59:26', '+/-': '33...","['1x Champions League Winner', '1x FIFA Club W...","['UEFA Supercup Winner', 'FIFA Club World Cup ...","['1', '1', '1', '2', '1', '1', '3', '4', '2', ...","{'value': '€400k', 'date': 'Oct 4, 2004'};{'va..."
2,2,Joško Gvardiol,"Jan 23, 2002 (22)",Zagreb,"1,85 m",Croatia,Defender - Centre-Back,left,Marjan Sisic,Manchester City,...,19.666667,1.0,"Mar 4, 2024",Premier League,€1.26bn,"{'Matches': '26', 'Goals': '59:26', '+/-': '33...","['1x Champions League Winner', '1x FIFA Club W...","['UEFA Supercup Winner', 'FIFA Club World Cup ...","['1', '1', '1', '2', '1', '1', '3', '4', '2', ...","{'value': '€300k', 'date': 'Sep 16, 2019'};{'v..."
3,3,Nathan Benjamin Aké,"Feb 18, 1995 (29)",Den Haag,"1,80 m",Netherlands Cote d'Ivoire,Defender - Centre-Back,left,Wasserman,Manchester City,...,21.250000,0.0,"May 27, 2023",Premier League,€1.26bn,"{'Matches': '26', 'Goals': '59:26', '+/-': '33...","['1x Champions League Winner', '1x FIFA Club W...","['UEFA Supercup Winner', 'FIFA Club World Cup ...","['1', '1', '1', '2', '1', '1', '3', '4', '2', ...","{'value': '€1.50m', 'date': 'Jul 25, 2013'};{'..."
4,4,Sergio Gómez Martín,"Sep 4, 2000 (23)",Badalona,"1,71 m",Spain,Defender - Left-Back,left,SEG,Manchester City,...,21.500000,0.0,"Nov 13, 2023",Premier League,€1.26bn,"{'Matches': '26', 'Goals': '59:26', '+/-': '33...","['1x Champions League Winner', '1x FIFA Club W...","['UEFA Supercup Winner', 'FIFA Club World Cup ...","['1', '1', '1', '2', '1', '1', '3', '4', '2', ...","{'value': '€2.00m', 'date': 'Apr 4, 2018'};{'v..."


### Сохранение файла

In [5]:
# Сохранение общего DataFrame в файл Excel 
result.to_excel("result.xlsx", index=True)